In [4]:
import pandas as pd
import numpy as np

from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder

import gc

In [5]:
train = pd.read_pickle("../data/train_agg.pkl",compression = "gzip")
test = pd.read_pickle("../data/test_agg.pkl",compression = "gzip")

### 범주형 변수 분류

In [6]:
cat_features = [
    "B_30",
    "B_38",
    "D_114",
    "D_116",
    "D_117",
    "D_120",
    "D_126",
    "D_63",
    "D_64",
    "D_66",
    "D_68"
]
cat_features = [f"{cf}_last" for cf in cat_features]
label_en = LabelEncoder()
for cf in cat_features: ### 두개의 63 64 last 값 category-> 라벨 인코딩
    train[cf] = label_en.fit_transform(train[cf])
    test[cf] = label_en.fit_transform(test[cf])

### 데이터 분류

In [7]:
X = train.drop('target',axis = 1)
y = train["target"]

In [8]:
X_train,X_test,y_train,y_test = train_test_split(X, y, random_state=2022)

In [9]:
X_train.shape,y_train.shape, X_test.shape,y_test.shape

((344184, 918), (344184,), (114729, 918), (114729,))

### 모델 학습

In [10]:
model = CatBoostClassifier(iterations=2000, random_state=2022, nan_mode='Min')

In [ ]:
%%time
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5,shuffle=True,random_state=2022)
print(cross_val_score(model,X_train,y_train, cv=kfold))


### 데이터 추출

In [ ]:
sub = pd.read_csv("../data/amex-default-prediction/sample_submission.csv")

In [14]:
y_preds = np.zeros(test.shape[0])
y_preds_ = np.zeros(test.shape[0])
y_preds = model.predict_proba(test)[:, 1]
y_preds_ = sub['prediction'].to_numpy()
cust_id = sub['customer_ID'].to_numpy()
blend_preds = (y_preds_ * 0.955 + y_preds * 0.045)

CatBoostError: There is no trained model to use predict(). Use fit() to train model. Then use this method.

In [ ]:
test["prediction"] = blend_preds
test["prediction"].to_csv(f"submission7.csv", index=True)